In [15]:
import numpy as np
import pandas as pd
import os
import rqdatac as rq

In [11]:
rqdatac.init('license', 'QhhTnnWjN5eWgeBzXChE-d4iqymtu4-6XIBIjcO4Oh339_i0W1LifdY980vuXMEq-Zxz03HTXeItx7cK-704N0Go8i7DERQjkXQIirqt6l4ZTVJIcJbC1Vh62p5npTupQN0F9SyXTIoE1UQPacRqhHUSWHQm4yNFQW2nHffVVlY=Th4zB2-UYw5Q76xBPpPrLCKIjzpVid9Wogpi8QP9Ju8M7ZRUXzA9V6TmKCWOlCrTPBWnA8HAjEL-3hM55cAac-hzVADGoxpfV4tHA79mMsjOCREOTXzQH6rqcqUuEDuzShSWFB069fRSetK3yzgeeRdqocHmJ4ATA3a3pCcAVpA=')

C:\Users\jiyic\anaconda3\lib\site-packages\rqdatac\client.py:236: UserWarning: Your account will be expired after  221 days. Please call us at 0755-22676337 to upgrade or purchase or renew your contract.
  warnings.warn("Your account will be expired after  {} days. "


In [13]:
rq.user.get_quota()

{'bytes_used': 2160020,
 'bytes_limit': 209715200.0,
 'remaining_days': 221,
 'license_type': 'TRIAL'}

In [51]:
import re 

stock_path = './data/stock_data/'
stock_names = os.listdir(path=stock_path)

format_stock_names = []
for name in stock_names:
    format_stock_names.append(name[:-4].strip().upper())

In [59]:
def normalize_code(symbol, pre_close=None):
    """
    归一化证券代码

    :param code 如000001
    :return 证券代码的全称 如000001.XSHE
    """
    XSHG = 'XSHG'
    SSE = 'XSHG'
    SH = 'XSHG'
    XSHE = 'XSHE'
    SZ = 'XSHE'
    SZE = 'XSHE'
    if (not isinstance(symbol, str)):
        return symbol

    if (symbol.startswith('SZ') and (len(symbol) == 8)):
        ret_normalize_code = '{}.{}'.format(symbol[2:8],SZ)
    elif (symbol.startswith('SH') and (len(symbol) == 8)):
        ret_normalize_code = '{}.{}'.format(symbol[2:8], SH)
    elif (symbol.startswith('00') and (len(symbol) == 6)):
        if ((pre_close is not None) and (pre_close > 2000)):
            # 推断是上证指数
            ret_normalize_code = '{}.{}'.format(symbol, SH)
        else:
            ret_normalize_code = '{}.{}'.format(symbol, SZ)
    elif ((symbol.startswith('399') or symbol.startswith('159') or \
        symbol.startswith('150')) and (len(symbol) == 6)):
        ret_normalize_code = '{}.{}'.format(symbol, SH)
    elif ((len(symbol) == 6) and (symbol.startswith('399') or \
        symbol.startswith('159') or symbol.startswith('150') or \
        symbol.startswith('16') or symbol.startswith('184801') or \
        symbol.startswith('201872'))):
        ret_normalize_code = '{}.{}'.format(symbol, SZ)
    elif ((len(symbol) == 6) and (symbol.startswith('50') or \
        symbol.startswith('51') or symbol.startswith('60') or \
        symbol.startswith('688') or symbol.startswith('900') or \
        (symbol == '751038'))):
        ret_normalize_code = '{}.{}'.format(symbol, SH)
    elif ((len(symbol) == 6) and (symbol[:3] in ['000', '001', '002', 
                                                 '200', '300'])):
        ret_normalize_code = '{}.{}'.format(symbol, SZ)
    else:
        print(symbol)
        ret_normalize_code = symbol

    return ret_normalize_code

In [56]:
format_stock_names[0]

'SH600000'

In [66]:
normalized_code = [normalize_code(name) for name in format_stock_names]
normalized_code[0]

'600000.XSHG'

Set time period = [20101231, 20201231] 十年回测

默认已经去除非交易日

In [76]:
len(normalized_code)

4560

In [102]:
# 检测回测时间内是否有ST退市股票
non_st_stock = rq.is_st_stock(normalized_code, '20100101','20201231')
# get column values i.e. stock code sorted
non_st_stock = non_st_stock.reindex(sorted(non_st_stock.columns), axis=1)

C:\Users\jiyic\anaconda3\lib\site-packages\rqdatac\validators.py:224: UserWarning: invalid order_book_id: 600849.XSHG
  warnings.warn("invalid order_book_id: {}".format(ob))


In [96]:
non_st_stock.shape

(2674, 4559)

In [103]:
count = non_st_stock.apply(pd.value_counts)

In [104]:
count
#这里肯定有问题，不可能所有股票都没退市，需要在外部网站上找个反例

,000001.XSHE,000002.XSHE,000003.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,...,688768.XSHG,688777.XSHG,688788.XSHG,688789.XSHG,688793.XSHG,688799.XSHG,688800.XSHG,688819.XSHG,688981.XSHG,689009.XSHG
False,2674.0,2674.0,2674.0,2330,2674.0,2674.0,2111,1877,2674.0,1460,...,2674.0,2674.0,2674.0,2674.0,2674.0,2674.0,2674.0,2674.0,2674.0,2674.0
True,NaN,NaN,NaN,344,NaN,NaN,563,797,NaN,1214,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
# all in ttm, not lyr 全部12个月滚动指标

#市净率
pb = rq.get_factor(normalized_code,'pb_ratio_ttm','20110101','20201231') 
#净资产收益率
roe = rq.get_factor(normalized_code,'return_on_equity_ttm','20110101','20201231') 
#总市值
mkt_cap = rq.get_factor(normalized_code,'market_cap_3','20110101','20201231')
#股息率
dividend_yield = rq.get_factor(normalized_code,'dividend_yield_ttm','20110101','20201231')
#账面市值比
bm = rq.get_factor(normalized_code,'book_to_market_ratio_ttm','20110101','20201231') 
#三日平均换手率
vol3 = rq.get_factor(normalized_code,'VOL3','20110101','20201231') 

C:\Users\jiyic\anaconda3\lib\site-packages\rqdatac\validators.py:224: UserWarning: invalid order_book_id: 600849.XSHG
  warnings.warn("invalid order_book_id: {}".format(ob))


In [122]:
pb.to_hdf('pb.h5', key='factor')

In [123]:
roe.to_hdf('roe.h5',key='factor')
mkt_cap.to_hdf('mkt_cap.h5',key='factor')
dividend_yield.to_hdf('dividend yield.h5', key='factor')
bm.to_hdf('bm.h5',key='factor')
vol3.to_hdf('vol3.h5',key='factor')

In [119]:
vol3

VOL3
order_book_id date                
000404.XSHE   2011-01-04       NaN
              2011-01-05  6.899900
              2011-01-06  7.909089
              2011-01-07  5.180172
              2011-01-10  3.398361
...                            ...
002465.XSHE   2020-12-25  1.276270
              2020-12-28  1.332772
              2020-12-29  1.394673
              2020-12-30  1.314515
              2020-12-31  1.302393

[11087488 rows x 1 columns]

In [120]:
sum(vol3.isna()['VOL3'])

4287947

In [117]:
vol3

return_on_equity_ttm
order_book_id date                            
000404.XSHE   2011-01-04              0.084820
              2011-01-05              0.084820
              2011-01-06              0.084820
              2011-01-07              0.084820
              2011-01-10              0.084820
...                                        ...
002465.XSHE   2020-12-25              0.057075
              2020-12-28              0.057075
              2020-12-29              0.057075
              2020-12-30              0.057075
              2020-12-31              0.057075

[11087488 rows x 1 columns]

In [116]:
# number of missing values
sum(roe.isna()['return_on_equity_ttm'])

4239827